# ML Hands-on Challenge - Data preparation and exploration

This notebook contains the data preparation and exploration for the ML Hands-on Challenge. The task will be to train a protein structure classification model using the CATH database.

## Loading things

In [1]:
import pandas as pd
import numpy as np
import sys
import glob
import os
import Bio.PDB.PDBParser
import py3Dmol
import warnings
from tqdm.notebook import tqdm

warnings.filterwarnings("ignore", message="Used element '.' for Atom")

# Get parent directory and add src/ to pythonpath
parent_dir = os.path.dirname(os.getcwd())
sys.path.append(parent_dir)

from src.config import CONSTANTS
from src.pdb import *

DATA_HOME = os.path.join(parent_dir, CONSTANTS.DATA_HOME)
architecture_names = CONSTANTS.ARCHITECTURE_NAMES

## Loading data

### Loading the `csv` dataframe with information about sequences

In [2]:
# Open the training data sequences and structure
data = pd.read_csv(f"{DATA_HOME}/cath_w_seqs_share.csv", index_col=0)
display(data)

# Get the schema of the data
data.info()

,cath_id,class,architecture,topology,superfamily,resolution_in_angstroms,pdb_id,sequences,cath_indices
0,2w3sB01,3,90,1170,50,2.60,2w3s,SVGKPLPHDSARAHVTGQARYLDDLPCPANTLHLAFGLSTEASAAI...,"[(2, 124)]"
1,3be3A00,2,30,30,320,2.04,3be3,QDFRPGVYRHYKGDHYLALGLARADETDEVVVVYTRLYARAGLPST...,"[(6, 81)]"
2,3zq4C03,3,10,20,580,3.00,3zq4,DIGNIVLRDRRILSEEGLVIVVVSIDMDDFKISAGPDLISRGFVIN...,"[(449, 555)]"
3,1peqA03,1,10,1650,20,2.80,1peq,DITFRLAKENAQMALFSPYDIQRRYGKPFGDIAISERYDELIADPH...,"[(294, 346)]"
4,1bdoA00,2,40,50,100,1.80,1bdo,EISGHIVRSPMVGTFYRTPSPDAKAFIEVGQKVNVGDTLCIVEAMK...,"[(77, 156)]"
...,...,...,...,...,...,...,...,...,...
6268,2yyiA02,2,40,110,10,1.66,2yyi,ATTHALTNPQVNRARPPSGQPDPYIPVGVVKQTEKGIVVRGARMTA...,"[(139, 266)]"
6269,4mo0A00,3,30,780,10,2.10,4mo0,EQKIKIYVTKRRFGKLMTIIEGFDTSVIDLKELAKKLKDICACGGT...,"[(24, 102)]"
6270,1vq8X00,3,10,440,10,2.20,1vq8,ERVVTIPLRDARAEPNHKRADKAMILIREHLAKHFSVDEDAVRLDP...,"[(7, 88)]"
6271,1ze3D00,3,10,20,410,1.84,1ze3,DLYFNPRFLLSRFENGQELPPGTYRVDIYLNNGYMATRDVTFNTGD...,"[(1, 125)]"


<class 'pandas.core.frame.DataFrame'>
Index: 6273 entries, 0 to 6272
Data columns (total 9 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   cath_id                  6273 non-null   object 
 1   class                    6273 non-null   int64  
 2   architecture             6273 non-null   int64  
 3   topology                 6273 non-null   int64  
 4   superfamily              6273 non-null   int64  
 5   resolution_in_angstroms  6273 non-null   float64
 6   pdb_id                   6273 non-null   object 
 7   sequences                6268 non-null   object 
 8   cath_indices             6268 non-null   object 
dtypes: float64(1), int64(4), object(4)
memory usage: 490.1+ KB


### Converting `cath_indices` to actual CATH indices using `eval`

In [3]:
# Apply eval to `cath_indices` with raising warning if catching error
def apply_eval_with_warning(x):
    try:
        return eval(x)
    except Exception as e:
        warnings.warn(f"Error occurred while evaluating {x}: {e}")
        return [(np.nan, np.nan)]


data["cath_indices_evaled"] = data["cath_indices"].apply(apply_eval_with_warning)
data

/var/folders/2h/yv02dgq91vd9r1zfwcgl5j3r0000gp/T/ipykernel_83746/681847689.py:6: UserWarning: Error occurred while evaluating nan: eval() arg 1 must be a string, bytes or code object
  warnings.warn(f"Error occurred while evaluating {x}: {e}")


,cath_id,class,architecture,topology,superfamily,resolution_in_angstroms,pdb_id,sequences,cath_indices,cath_indices_evaled
0,2w3sB01,3,90,1170,50,2.60,2w3s,SVGKPLPHDSARAHVTGQARYLDDLPCPANTLHLAFGLSTEASAAI...,"[(2, 124)]","[(2, 124)]"
1,3be3A00,2,30,30,320,2.04,3be3,QDFRPGVYRHYKGDHYLALGLARADETDEVVVVYTRLYARAGLPST...,"[(6, 81)]","[(6, 81)]"
2,3zq4C03,3,10,20,580,3.00,3zq4,DIGNIVLRDRRILSEEGLVIVVVSIDMDDFKISAGPDLISRGFVIN...,"[(449, 555)]","[(449, 555)]"
3,1peqA03,1,10,1650,20,2.80,1peq,DITFRLAKENAQMALFSPYDIQRRYGKPFGDIAISERYDELIADPH...,"[(294, 346)]","[(294, 346)]"
4,1bdoA00,2,40,50,100,1.80,1bdo,EISGHIVRSPMVGTFYRTPSPDAKAFIEVGQKVNVGDTLCIVEAMK...,"[(77, 156)]","[(77, 156)]"
...,...,...,...,...,...,...,...,...,...,...
6268,2yyiA02,2,40,110,10,1.66,2yyi,ATTHALTNPQVNRARPPSGQPDPYIPVGVVKQTEKGIVVRGARMTA...,"[(139, 266)]","[(139, 266)]"
6269,4mo0A00,3,30,780,10,2.10,4mo0,EQKIKIYVTKRRFGKLMTIIEGFDTSVIDLKELAKKLKDICACGGT...,"[(24, 102)]","[(24, 102)]"
6270,1vq8X00,3,10,440,10,2.20,1vq8,ERVVTIPLRDARAEPNHKRADKAMILIREHLAKHFSVDEDAVRLDP...,"[(7, 88)]","[(7, 88)]"
6271,1ze3D00,3,10,20,410,1.84,1ze3,DLYFNPRFLLSRFENGQELPPGTYRVDIYLNNGYMATRDVTFNTGD...,"[(1, 125)]","[(1, 125)]"


### Loading the information from sequences

#### Asserting that `cath_id` in the `data` corresponds to files in `pdb_share` folder

In [4]:
# List how many files are in {DATA}/data/pdb_share
pdb_share_dir = os.path.join(DATA_HOME, "pdb_share")
file_paths = glob.glob(os.path.join(pdb_share_dir, "*"))

# Get a list of file names
file_names = [os.path.basename(file_path) for file_path in file_paths]

# Assert that set of `cath_id` from `data` is equal to the set of `file_names`
assert set(data["cath_id"]) == set(file_names)

### Loading the structures from pdb files

In [5]:
data = data.assign(
    **{
        "pdb_sequences": data["cath_id"].apply(
            lambda x: get_sequence_from_pdb_file(f"{DATA_HOME}/pdb_share/{x}")
        ),
        "pdb_cath_indices": data["cath_id"].apply(
            lambda x: get_res_list_from_pdb_file(f"{DATA_HOME}/pdb_share/{x}")
        ),
    }
)
data

/Users/I0550745/miniconda3/envs/protein-pred/lib/python3.10/site-packages/Bio/PDB/Atom.py:232: PDBConstructionWarning: Used element 'SE' for Atom (name=SE) with given element ''
  warnings.warn(msg, PDBConstructionWarning)
/Users/I0550745/miniconda3/envs/protein-pred/lib/python3.10/site-packages/Bio/PDB/StructureBuilder.py:273: PDBConstructionWarning: WARNING: disordered atom found with blank altloc before line 817.

  warnings.warn(
/Users/I0550745/miniconda3/envs/protein-pred/lib/python3.10/site-packages/Bio/PDB/Atom.py:232: PDBConstructionWarning: Used element 'SE' for Atom (name=SE) with given element ''
  warnings.warn(msg, PDBConstructionWarning)
/Users/I0550745/miniconda3/envs/protein-pred/lib/python3.10/site-packages/Bio/PDB/StructureBuilder.py:273: PDBConstructionWarning: WARNING: disordered atom found with blank altloc before line 817.

  warnings.warn(


,cath_id,class,architecture,topology,superfamily,resolution_in_angstroms,pdb_id,sequences,cath_indices,cath_indices_evaled,pdb_sequences,pdb_cath_indices
0,2w3sB01,3,90,1170,50,2.60,2w3s,SVGKPLPHDSARAHVTGQARYLDDLPCPANTLHLAFGLSTEASAAI...,"[(2, 124)]","[(2, 124)]",SVGKPLPHDSARAHVTGQARYLDDLPCPANTLHLAFGLSTEASAAI...,"[(2, 124)]"
1,3be3A00,2,30,30,320,2.04,3be3,QDFRPGVYRHYKGDHYLALGLARADETDEVVVVYTRLYARAGLPST...,"[(6, 81)]","[(6, 81)]",QDFRPGVYRHYKGDHYLALGLARADETDEVVVVYTRLYARAGLPST...,"[(6, 49), (51, 81)]"
2,3zq4C03,3,10,20,580,3.00,3zq4,DIGNIVLRDRRILSEEGLVIVVVSIDMDDFKISAGPDLISRGFVIN...,"[(449, 555)]","[(449, 555)]",DIGNIVLRDRRILSEEGLVIVVVSIDMDDFKISAGPDLISRGFVIN...,"[(449, 492), (501, 555)]"
3,1peqA03,1,10,1650,20,2.80,1peq,DITFRLAKENAQMALFSPYDIQRRYGKPFGDIAISERYDELIADPH...,"[(294, 346)]","[(294, 346)]",DITFRLAKENAQMALFSPYDIQRRYGKPFGDIAISERYDELIADPH...,"[(294, 346)]"
4,1bdoA00,2,40,50,100,1.80,1bdo,EISGHIVRSPMVGTFYRTPSPDAKAFIEVGQKVNVGDTLCIVEAMK...,"[(77, 156)]","[(77, 156)]",EISGHIVRSPMVGTFYRTPSPDAKAFIEVGQKVNVGDTLCIVEAMK...,"[(77, 156)]"
...,...,...,...,...,...,...,...,...,...,...,...,...
6268,2yyiA02,2,40,110,10,1.66,2yyi,ATTHALTNPQVNRARPPSGQPDPYIPVGVVKQTEKGIVVRGARMTA...,"[(139, 266)]","[(139, 266)]",ATTHALTNPQVNRARPPSGQPDPYIPVGVVKQTEKGIVVRGARMTA...,"[(139, 196), (199, 266)]"
6269,4mo0A00,3,30,780,10,2.10,4mo0,EQKIKIYVTKRRFGKLMTIIEGFDTSVIDLKELAKKLKDICACGGT...,"[(24, 102)]","[(24, 102)]",EQKIKIYVTKRRFGKLMTIIEGFDTSVIDLKELAKKLKDICACGGT...,"[(24, 102)]"
6270,1vq8X00,3,10,440,10,2.20,1vq8,ERVVTIPLRDARAEPNHKRADKAMILIREHLAKHFSVDEDAVRLDP...,"[(7, 88)]","[(7, 88)]",ERVVTIPLRDARAEPNHKRADKAMILIREHLAKHFSVDEDAVRLDP...,"[(7, 88)]"
6271,1ze3D00,3,10,20,410,1.84,1ze3,DLYFNPRFLLSRFENGQELPPGTYRVDIYLNNGYMATRDVTFNTGD...,"[(1, 125)]","[(1, 125)]",DLYFNPRFLLSRFENGQELPPGTYRVDIYLNNGYMATRDVTFNTGD...,"[(1, 9), (19, 125)]"


### Asserting that `sequences` and `pdb_sequences` are the same

In [6]:
mismatch_data = data[data["pdb_sequences"] != data["sequences"]]
mismatch_data

,cath_id,class,architecture,topology,superfamily,resolution_in_angstroms,pdb_id,sequences,cath_indices,cath_indices_evaled,pdb_sequences,pdb_cath_indices
7,2gnxA01,1,10,3450,30,2.45,2gnx,NaN,"[(101, 295), (431, 440)]","[(101, 295), (431, 440)]",XXXXXXXXXXXXXXXXXXXXXPHLSEQLCFFVQAREIADFYEKYAL...,"[(101, 121), (123, 136), (138, 145), (147, 172..."
2827,2xskA00,2,60,40,2420,1.70,2xsk,NaN,"[(3, 97)]","[(3, 97)]",SSQITFNTTQQGDYTIIPEVTLTQSXLXRVQILSLREGSSGQSQTK...,"[(3, 15), (17, 97)]"
4668,4dgwC00,2,60,40,2690,3.11,4dgw,NaN,"[(101, 253)]","[(101, 253)]",XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXGSVGAIQVNY...,"[(101, 136), (149, 152), (154, 225), (227, 253)]"
5776,1nthA00,3,20,20,460,1.55,1nth,NaN,"[(2, 458)]","[(2, 458)]",TFRKSFDCYDFYDRAKVGEKCTQDDWDLMKIPMKAMELKQKYGLDF...,"[(2, 458)]"
6010,3pieA02,3,30,1370,250,2.90,3pie,NaN,"[(379, 487)]","[(379, 487)]",GKKLLMKQQKKLIGAVKPWLLKTVQRKVTSDADFEIFPLEDKELVR...,"[(379, 408), (412, 462), (469, 487)]"


We can see that there are some mismatch sequences when comparing local pdb files and `sequences` from the `csv` file. Apparently, it is due to unusual / non-standard amino acids. We will remove these sequences from the dataframe.

In [7]:
# Dropping the mismatched data
seq_filtered_data = data.drop(mismatch_data.index)

### Asserting that `cath_indices` and `pdb_cath_indices` are the same

In [8]:
mismatch_rows = seq_filtered_data[
    seq_filtered_data["cath_indices_evaled"] != seq_filtered_data["pdb_cath_indices"]
]
mismatch_rows

,cath_id,class,architecture,topology,superfamily,resolution_in_angstroms,pdb_id,sequences,cath_indices,cath_indices_evaled,pdb_sequences,pdb_cath_indices
1,3be3A00,2,30,30,320,2.04,3be3,QDFRPGVYRHYKGDHYLALGLARADETDEVVVVYTRLYARAGLPST...,"[(6, 81)]","[(6, 81)]",QDFRPGVYRHYKGDHYLALGLARADETDEVVVVYTRLYARAGLPST...,"[(6, 49), (51, 81)]"
2,3zq4C03,3,10,20,580,3.00,3zq4,DIGNIVLRDRRILSEEGLVIVVVSIDMDDFKISAGPDLISRGFVIN...,"[(449, 555)]","[(449, 555)]",DIGNIVLRDRRILSEEGLVIVVVSIDMDDFKISAGPDLISRGFVIN...,"[(449, 492), (501, 555)]"
6,1aqcA00,2,30,29,30,2.30,1aqc,EDLIDGIIFAANYLGSTQLLSDKTPSKNVRQAQEAVSRIKAQKLTE...,"[(324, 488)]","[(324, 488)]",EDLIDGIIFAANYLGSTQLLSDKTPSKNVRQAQEAVSRIKAQKLTE...,"[(324, 353), (356, 365), (367, 370), (386, 407..."
12,3i9v600,3,40,50,12280,3.10,3i9v,EREGILFTTLEKLVAWGRSNSLWPATFGLACCAIEMMASTDARQAD...,"[(15, 175)]","[(15, 175)]",EREGILFTTLEKLVAWGRSNSLWPATFGLACCAIEMMASTDARQAD...,"[(15, 57), (74, 175)]"
13,3rylA01,1,20,120,1210,3.10,3ryl,GHRLLSEDLFKQSPKLSEQELDELANNLADYLFQAADIDISTEEKN...,"[(244, 283), (385, 481)]","[(244, 283), (385, 481)]",GHRLLSEDLFKQSPKLSEQELDELANNLADYLFQAADIDISTEEKN...,"[(244, 245), (247, 283), (385, 411), (413, 448..."
...,...,...,...,...,...,...,...,...,...,...,...,...
6266,1n99A02,2,30,42,10,1.94,1n99,RTITHKDSTGHVGFIFKNGKITSIVKDSSAARNGLLTEHNICEING...,"[(197, 269)]","[(197, 269)]",RTITHKDSTGHVGFIFKNGKITSIVKDSSAARNGLLTEHNICEING...,"[(197, 200), (202, 269)]"
6267,1jceA03,3,90,640,10,2.10,1jce,AGDEDEAIVQYVRETYRVAIGERTAERVKIEIGNVFPSKENDELET...,"[(178, 253)]","[(178, 253)]",AGDEDEAIVQYVRETYRVAIGERTAERVKIEIGNVFPSKENDELET...,"[(178, 181), (183, 253)]"
6268,2yyiA02,2,40,110,10,1.66,2yyi,ATTHALTNPQVNRARPPSGQPDPYIPVGVVKQTEKGIVVRGARMTA...,"[(139, 266)]","[(139, 266)]",ATTHALTNPQVNRARPPSGQPDPYIPVGVVKQTEKGIVVRGARMTA...,"[(139, 196), (199, 266)]"
6271,1ze3D00,3,10,20,410,1.84,1ze3,DLYFNPRFLLSRFENGQELPPGTYRVDIYLNNGYMATRDVTFNTGD...,"[(1, 125)]","[(1, 125)]",DLYFNPRFLLSRFENGQELPPGTYRVDIYLNNGYMATRDVTFNTGD...,"[(1, 9), (19, 125)]"


We see that around 2.7k sequences have gaps in the sequences that are depicted in the `pdb_cath_indices` but not in the `cath_indices`. It would be interesting to have a look at the sequence lengths.

### Calculating the length of the sequences

In [9]:
filtered_data = seq_filtered_data.assign(
    **{
        "sequences_len": seq_filtered_data["sequences"].apply(len),
        "pdb_sequences_len": seq_filtered_data["pdb_sequences"].apply(len),
        "cath_indices_len": seq_filtered_data["cath_indices_evaled"].apply(
            lambda lst: np.sum([end - start + 1 for start, end in lst])
        ),
        "pdb_cath_indices_len": seq_filtered_data["pdb_cath_indices"].apply(
            lambda lst: np.sum([end - start + 1 for start, end in lst])
        ),
    }
)
filtered_data

,cath_id,class,architecture,topology,superfamily,resolution_in_angstroms,pdb_id,sequences,cath_indices,cath_indices_evaled,pdb_sequences,pdb_cath_indices,sequences_len,pdb_sequences_len,cath_indices_len,pdb_cath_indices_len
0,2w3sB01,3,90,1170,50,2.60,2w3s,SVGKPLPHDSARAHVTGQARYLDDLPCPANTLHLAFGLSTEASAAI...,"[(2, 124)]","[(2, 124)]",SVGKPLPHDSARAHVTGQARYLDDLPCPANTLHLAFGLSTEASAAI...,"[(2, 124)]",123,123,123.0,123
1,3be3A00,2,30,30,320,2.04,3be3,QDFRPGVYRHYKGDHYLALGLARADETDEVVVVYTRLYARAGLPST...,"[(6, 81)]","[(6, 81)]",QDFRPGVYRHYKGDHYLALGLARADETDEVVVVYTRLYARAGLPST...,"[(6, 49), (51, 81)]",75,75,76.0,75
2,3zq4C03,3,10,20,580,3.00,3zq4,DIGNIVLRDRRILSEEGLVIVVVSIDMDDFKISAGPDLISRGFVIN...,"[(449, 555)]","[(449, 555)]",DIGNIVLRDRRILSEEGLVIVVVSIDMDDFKISAGPDLISRGFVIN...,"[(449, 492), (501, 555)]",99,99,107.0,99
3,1peqA03,1,10,1650,20,2.80,1peq,DITFRLAKENAQMALFSPYDIQRRYGKPFGDIAISERYDELIADPH...,"[(294, 346)]","[(294, 346)]",DITFRLAKENAQMALFSPYDIQRRYGKPFGDIAISERYDELIADPH...,"[(294, 346)]",53,53,53.0,53
4,1bdoA00,2,40,50,100,1.80,1bdo,EISGHIVRSPMVGTFYRTPSPDAKAFIEVGQKVNVGDTLCIVEAMK...,"[(77, 156)]","[(77, 156)]",EISGHIVRSPMVGTFYRTPSPDAKAFIEVGQKVNVGDTLCIVEAMK...,"[(77, 156)]",80,80,80.0,80
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6268,2yyiA02,2,40,110,10,1.66,2yyi,ATTHALTNPQVNRARPPSGQPDPYIPVGVVKQTEKGIVVRGARMTA...,"[(139, 266)]","[(139, 266)]",ATTHALTNPQVNRARPPSGQPDPYIPVGVVKQTEKGIVVRGARMTA...,"[(139, 196), (199, 266)]",126,126,128.0,126
6269,4mo0A00,3,30,780,10,2.10,4mo0,EQKIKIYVTKRRFGKLMTIIEGFDTSVIDLKELAKKLKDICACGGT...,"[(24, 102)]","[(24, 102)]",EQKIKIYVTKRRFGKLMTIIEGFDTSVIDLKELAKKLKDICACGGT...,"[(24, 102)]",79,79,79.0,79
6270,1vq8X00,3,10,440,10,2.20,1vq8,ERVVTIPLRDARAEPNHKRADKAMILIREHLAKHFSVDEDAVRLDP...,"[(7, 88)]","[(7, 88)]",ERVVTIPLRDARAEPNHKRADKAMILIREHLAKHFSVDEDAVRLDP...,"[(7, 88)]",82,82,82.0,82
6271,1ze3D00,3,10,20,410,1.84,1ze3,DLYFNPRFLLSRFENGQELPPGTYRVDIYLNNGYMATRDVTFNTGD...,"[(1, 125)]","[(1, 125)]",DLYFNPRFLLSRFENGQELPPGTYRVDIYLNNGYMATRDVTFNTGD...,"[(1, 9), (19, 125)]",116,116,125.0,116


Great! We see how `path_cath_indices_len` almost matches the `sequences_len` and `pdb_sequences_len`. Let's confirm that.

In [10]:
mismatched_rows = filtered_data[
    filtered_data["pdb_cath_indices_len"] != filtered_data["sequences_len"]
]
display(mismatched_rows)

mismatched_rows = filtered_data[
    filtered_data["pdb_cath_indices_len"] != filtered_data["pdb_sequences_len"]
]
mismatched_rows

,cath_id,class,architecture,topology,superfamily,resolution_in_angstroms,pdb_id,sequences,cath_indices,cath_indices_evaled,pdb_sequences,pdb_cath_indices,sequences_len,pdb_sequences_len,cath_indices_len,pdb_cath_indices_len


,cath_id,class,architecture,topology,superfamily,resolution_in_angstroms,pdb_id,sequences,cath_indices,cath_indices_evaled,pdb_sequences,pdb_cath_indices,sequences_len,pdb_sequences_len,cath_indices_len,pdb_cath_indices_len


Two empty sets! Great so it means:
- The `cath_indices` contains the global boundaries of the CATH domains or supersets of the local boundaries (i.e. the coordinates of the whole sequence)
- The `pdb_cath_indices` contains the local boundaries of the CATH domains (i.e. the coordinates of each piece)

Let's rename the columns to make it clearer.

In [11]:
cols_to_keep = [
    "cath_id",
    "pdb_id",
    "class",
    "architecture",
    "topology",
    "superfamily",
    "resolution_in_angstroms",
    "sequence",
    "piece_edges",
    "num_residues",
    "domain_edges",
    "total_num_residues",
]

proc_data = filtered_data.rename(
    columns={
        "pdb_cath_indices": "piece_edges",  # edge indices of each piece
        "sequences_len": "num_residues",  # present/resolved residues in the sequence
        "sequences": "sequence",
    }
).assign(
    **{
        "domain_edges": lambda df: df["piece_edges"].apply(
            lambda lst: [lst[0][0], lst[-1][1]]
        ),
        "total_num_residues": lambda df: df["domain_edges"].apply(
            lambda lst: lst[1] - lst[0] + 1
        ),
    }
)[cols_to_keep]

proc_data

,cath_id,pdb_id,class,architecture,topology,superfamily,resolution_in_angstroms,sequence,piece_edges,num_residues,domain_edges,total_num_residues
0,2w3sB01,2w3s,3,90,1170,50,2.60,SVGKPLPHDSARAHVTGQARYLDDLPCPANTLHLAFGLSTEASAAI...,"[(2, 124)]",123,"[2, 124]",123
1,3be3A00,3be3,2,30,30,320,2.04,QDFRPGVYRHYKGDHYLALGLARADETDEVVVVYTRLYARAGLPST...,"[(6, 49), (51, 81)]",75,"[6, 81]",76
2,3zq4C03,3zq4,3,10,20,580,3.00,DIGNIVLRDRRILSEEGLVIVVVSIDMDDFKISAGPDLISRGFVIN...,"[(449, 492), (501, 555)]",99,"[449, 555]",107
3,1peqA03,1peq,1,10,1650,20,2.80,DITFRLAKENAQMALFSPYDIQRRYGKPFGDIAISERYDELIADPH...,"[(294, 346)]",53,"[294, 346]",53
4,1bdoA00,1bdo,2,40,50,100,1.80,EISGHIVRSPMVGTFYRTPSPDAKAFIEVGQKVNVGDTLCIVEAMK...,"[(77, 156)]",80,"[77, 156]",80
...,...,...,...,...,...,...,...,...,...,...,...,...
6268,2yyiA02,2yyi,2,40,110,10,1.66,ATTHALTNPQVNRARPPSGQPDPYIPVGVVKQTEKGIVVRGARMTA...,"[(139, 196), (199, 266)]",126,"[139, 266]",128
6269,4mo0A00,4mo0,3,30,780,10,2.10,EQKIKIYVTKRRFGKLMTIIEGFDTSVIDLKELAKKLKDICACGGT...,"[(24, 102)]",79,"[24, 102]",79
6270,1vq8X00,1vq8,3,10,440,10,2.20,ERVVTIPLRDARAEPNHKRADKAMILIREHLAKHFSVDEDAVRLDP...,"[(7, 88)]",82,"[7, 88]",82
6271,1ze3D00,1ze3,3,10,20,410,1.84,DLYFNPRFLLSRFENGQELPPGTYRVDIYLNNGYMATRDVTFNTGD...,"[(1, 9), (19, 125)]",116,"[1, 125]",125


## [Optional] Recovering missing residues

Let us look at the `piece_edges` and understand the prevalence of gaps.

In [12]:
proc_data = proc_data.assign(
    **{
        'num_gaps': proc_data['piece_edges'].apply(lambda x: len(x) - 1),
    }
)

proc_data['num_gaps'].value_counts()

num_gaps
0     3037
1     1433
2      680
3      406
4      236
5      164
6      105
7       62
8       46
9       37
11      21
10      18
12       6
15       5
14       5
13       4
19       1
16       1
18       1
Name: count, dtype: int64

We will try to download the protein structures from the PDB database and see if we can recover the missing residues.

In [13]:
# Extracting sequences with missing residues
miss_residue_pdbs = proc_data[proc_data['num_gaps'] > 0]
miss_residue_pdbs

,cath_id,pdb_id,class,architecture,topology,superfamily,resolution_in_angstroms,sequence,piece_edges,num_residues,domain_edges,total_num_residues,num_gaps
1,3be3A00,3be3,2,30,30,320,2.04,QDFRPGVYRHYKGDHYLALGLARADETDEVVVVYTRLYARAGLPST...,"[(6, 49), (51, 81)]",75,"[6, 81]",76,1
2,3zq4C03,3zq4,3,10,20,580,3.00,DIGNIVLRDRRILSEEGLVIVVVSIDMDDFKISAGPDLISRGFVIN...,"[(449, 492), (501, 555)]",99,"[449, 555]",107,1
6,1aqcA00,1aqc,2,30,29,30,2.30,EDLIDGIIFAANYLGSTQLLSDKTPSKNVRQAQEAVSRIKAQKLTE...,"[(324, 353), (356, 365), (367, 370), (386, 407...",121,"[324, 488]",165,7
12,3i9v600,3i9v,3,40,50,12280,3.10,EREGILFTTLEKLVAWGRSNSLWPATFGLACCAIEMMASTDARQAD...,"[(15, 57), (74, 175)]",145,"[15, 175]",161,1
13,3rylA01,3ryl,1,20,120,1210,3.10,GHRLLSEDLFKQSPKLSEQELDELANNLADYLFQAADIDISTEEKN...,"[(244, 245), (247, 283), (385, 411), (413, 448...",134,"[244, 481]",238,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...
6266,1n99A02,1n99,2,30,42,10,1.94,RTITHKDSTGHVGFIFKNGKITSIVKDSSAARNGLLTEHNICEING...,"[(197, 200), (202, 269)]",72,"[197, 269]",73,1
6267,1jceA03,1jce,3,90,640,10,2.10,AGDEDEAIVQYVRETYRVAIGERTAERVKIEIGNVFPSKENDELET...,"[(178, 181), (183, 253)]",75,"[178, 253]",76,1
6268,2yyiA02,2yyi,2,40,110,10,1.66,ATTHALTNPQVNRARPPSGQPDPYIPVGVVKQTEKGIVVRGARMTA...,"[(139, 196), (199, 266)]",126,"[139, 266]",128,1
6271,1ze3D00,1ze3,3,10,20,410,1.84,DLYFNPRFLLSRFENGQELPPGTYRVDIYLNNGYMATRDVTFNTGD...,"[(1, 9), (19, 125)]",116,"[1, 125]",125,1


3.2k sequences have gaps!

<font color='red'>The cell below can run for around ~30 minutes as it downloads all the sequences with gaps!</font>

In [14]:
LOAD_PDB = False  # Set to True to download PDB files

if LOAD_PDB:
    for i, row in tqdm(miss_residue_pdbs.iterrows(), total=miss_residue_pdbs.shape[0]):
        download_pdb(row['pdb_id'], f"{DATA_HOME}/pdb_downloaded")

### Matching the newly downloaded sequences and the original ones

Now as we downloaded the new sequences, let's see if we can match them.

In [15]:
def get_sequence_from_pdb_file(pdb_filename, nonstandard_warnings=False):
    """
    Retrieves the protein sequence from a PDB file.

    Args:
        pdb_filename (str): The path to the PDB file.
        nonstandard_warnings (bool, optional): If True, warning messages for nonstandard residues.
            Defaults to False.

    Returns:
        str: The protein sequence extracted from the PDB file.
    """
    pdb_parser = Bio.PDB.PDBParser()
    structure = pdb_parser.get_structure(pdb_filename, pdb_filename)
    assert len(structure) == 1

    seq = []

    for model in structure:
        for chain in model:
            for residue in chain:
                if residue.get_id()[0] == " ":  # This checks if it's a standard residue
                    seq.append(protein_letters_3to1[residue.get_resname()])
                else:
                    if nonstandard_warnings:
                        print("nonstandard", residue.get_id())

    return "".join(seq)

In [16]:
random_pdb = '1ze3'

print('The sequence with gaps info:')
df = miss_residue_pdbs[miss_residue_pdbs['pdb_id'] == random_pdb][['cath_id', 'pdb_id', 'domain_edges', 'piece_edges', 'sequence']]
display(df)

print('Original sequence:')
orig_seq = df['sequence'].values[0]
print(orig_seq)

print('Sequence with gaps:')
new_seq = get_sequence_from_pdb_file(f"{DATA_HOME}/pdb_downloaded/{random_pdb}.pdb")
print(new_seq)

The sequence with gaps info:


,cath_id,pdb_id,domain_edges,piece_edges,sequence
6271,1ze3D00,1ze3,"[1, 125]","[(1, 9), (19, 125)]",DLYFNPRFLLSRFENGQELPPGTYRVDIYLNNGYMATRDVTFNTGD...


Original sequence:
DLYFNPRFLLSRFENGQELPPGTYRVDIYLNNGYMATRDVTFNTGDSEQGIVPCLTRAQLASMGLNTASVAGMNLLADDACVPLTTMVQDATAHLDVGQQRLNLTIPQAFMSNRAR
Sequence with gaps:
GVALGATRVIYPAGQKQEQLAVTNNDENSTYLIQSWVENADGVKDGRFIVTPPLFAMKGKKENTLRILDATNNQLPQDRESLFWMNVKAIPSMDKSKLTENTLQLAIISRIKLYYRPAKLALPPDQAAEKLRFRRSANSLTLINPTPYYLTVTELNAGTRVLENALVPPMGESTVKLPSDAGSNITYRTINDYGALTPKMTGVMETGGCDVSARDVTVTLPDYPGSVPIPLTVYCAKSQNLGYYLSGTTADAGNSIFTNTASFSPAQGVGVQLTRNGTIIPANNTVSLGAVGTSAVSLGLTANYARTGGQVTAGNVQSIIGVTFVYQDLYFNPRFLLSRFENGQELPPGTYRVDIYLNNGYMATRDVTFNTGDSEQGIVPCLTRAQLASMGLNTASVAGMNLLADDACVPLTTMVQDATAHLDVGQQRLNLTIPQAFMSNRAR


/Users/I0550745/miniconda3/envs/protein-pred/lib/python3.10/site-packages/Bio/PDB/StructureBuilder.py:89: PDBConstructionWarning: WARNING: Chain C is discontinuous at line 3873.
  warnings.warn(
/Users/I0550745/miniconda3/envs/protein-pred/lib/python3.10/site-packages/Bio/PDB/StructureBuilder.py:89: PDBConstructionWarning: WARNING: Chain H is discontinuous at line 3905.
  warnings.warn(
/Users/I0550745/miniconda3/envs/protein-pred/lib/python3.10/site-packages/Bio/PDB/StructureBuilder.py:89: PDBConstructionWarning: WARNING: Chain D is discontinuous at line 3917.
  warnings.warn(
/Users/I0550745/miniconda3/envs/protein-pred/lib/python3.10/site-packages/Bio/PDB/StructureBuilder.py:89: PDBConstructionWarning: WARNING: Chain C is discontinuous at line 3921.
  warnings.warn(
/Users/I0550745/miniconda3/envs/protein-pred/lib/python3.10/site-packages/Bio/PDB/StructureBuilder.py:89: PDBConstructionWarning: WARNING: Chain H is discontinuous at line 4143.
  warnings.warn(
/Users/I0550745/miniconda

We can see that the second sequence is much longer.. So we need to align them!

In [17]:
from Bio import pairwise2
from Bio.pairwise2 import format_alignment

def align_sequences(s1, s2):
    alignments = pairwise2.align.globalxx(s1, s2)
    print(format_alignment(*alignments[0]))

align_sequences(orig_seq, new_seq)

-------------------------D-----L---------------------------------------------------------------------------------Y------------------F----N------P--------------R------------------------------------------------------------------------------------------------F---------------L--------------L-----S-----------R-----------------F----------------ENGQELPPGTYRVDIYLNNGYMATRDVTFNTGDSEQGIVPCLTRAQLASMGLNTASVAGMNLLADDACVPLTTMVQDATAHLDVGQQRLNLTIPQAFMSNRAR
                         |     |                                                                                 |                  |    |      |              |                                                                                                |               |              |     |           |                 |                |||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||
GVALGATRVIYPAGQKQEQLAVTNNDENSTYLIQSWVENADGVKDGRFIVTPPLFAMKGKKENTLRILDATNNQLPQDRESLFWMNVKAIPSMDKSKLTENTLQLAIISRIK

/Users/I0550745/miniconda3/envs/protein-pred/lib/python3.10/site-packages/Bio/pairwise2.py:278: BiopythonDeprecationWarning: Bio.pairwise2 has been deprecated, and we intend to remove it in a future release of Biopython. As an alternative, please consider using Bio.Align.PairwiseAligner as a replacement, and contact the Biopython developers if you still need the Bio.pairwise2 module.
  warnings.warn(


We can see that the right part of the sequence perfectly aligns.

*However, it seems like a lot of work to match new sequences with old ones so we will take the sequences as they are.*

## Saving the data

### Let's create the target class, then save the data

We will save `target` as merged `{class}.{architecture}`.

In [18]:
out_data = proc_data.assign(
    target=lambda df: df["class"].astype(str) + "." + df["architecture"].astype(str)
).reset_index(drop=True)

print('Target distribution')
display(out_data['target'].value_counts())

out_data.to_csv(f"{DATA_HOME}/data_processed.csv")
out_data

Target distribution


target
2.30    640
3.90    634
2.40    634
3.40    628
2.60    628
1.10    626
1.20    626
3.30    626
3.10    614
3.20    612
Name: count, dtype: int64

,cath_id,pdb_id,class,architecture,topology,superfamily,resolution_in_angstroms,sequence,piece_edges,num_residues,domain_edges,total_num_residues,num_gaps,target
0,2w3sB01,2w3s,3,90,1170,50,2.60,SVGKPLPHDSARAHVTGQARYLDDLPCPANTLHLAFGLSTEASAAI...,"[(2, 124)]",123,"[2, 124]",123,0,3.90
1,3be3A00,3be3,2,30,30,320,2.04,QDFRPGVYRHYKGDHYLALGLARADETDEVVVVYTRLYARAGLPST...,"[(6, 49), (51, 81)]",75,"[6, 81]",76,1,2.30
2,3zq4C03,3zq4,3,10,20,580,3.00,DIGNIVLRDRRILSEEGLVIVVVSIDMDDFKISAGPDLISRGFVIN...,"[(449, 492), (501, 555)]",99,"[449, 555]",107,1,3.10
3,1peqA03,1peq,1,10,1650,20,2.80,DITFRLAKENAQMALFSPYDIQRRYGKPFGDIAISERYDELIADPH...,"[(294, 346)]",53,"[294, 346]",53,0,1.10
4,1bdoA00,1bdo,2,40,50,100,1.80,EISGHIVRSPMVGTFYRTPSPDAKAFIEVGQKVNVGDTLCIVEAMK...,"[(77, 156)]",80,"[77, 156]",80,0,2.40
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6263,2yyiA02,2yyi,2,40,110,10,1.66,ATTHALTNPQVNRARPPSGQPDPYIPVGVVKQTEKGIVVRGARMTA...,"[(139, 196), (199, 266)]",126,"[139, 266]",128,1,2.40
6264,4mo0A00,4mo0,3,30,780,10,2.10,EQKIKIYVTKRRFGKLMTIIEGFDTSVIDLKELAKKLKDICACGGT...,"[(24, 102)]",79,"[24, 102]",79,0,3.30
6265,1vq8X00,1vq8,3,10,440,10,2.20,ERVVTIPLRDARAEPNHKRADKAMILIREHLAKHFSVDEDAVRLDP...,"[(7, 88)]",82,"[7, 88]",82,0,3.10
6266,1ze3D00,1ze3,3,10,20,410,1.84,DLYFNPRFLLSRFENGQELPPGTYRVDIYLNNGYMATRDVTFNTGD...,"[(1, 9), (19, 125)]",116,"[1, 125]",125,1,3.10
